In [2]:
# загружаем необходимые библиотеки
import warnings

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

# загружаем датафрейм
df = pd.read_csv("discountuplift.csv", sep="\t")

df["old_target"] = (df["target_class"] % 2).apply(int)

feature_cols = [
    "recency",
    "history",
    "used_discount",
    "used_bogo",
    "is_referral",
    "zip_code_Rural",
    "zip_code_Surburban",
    "zip_code_Urban",
    "channel_Multichannel",
    "channel_Phone",
    "channel_Web",
]
target_col = "old_target"
treatment_col = "treatment"

# разобъём нашу выборку на тестовую и валидационную
df_train, df_test = train_test_split(
    df, stratify=df[[treatment_col, target_col]], random_state=1, test_size=0.25
)

In [3]:
from causalml.inference.tree import UpliftTreeClassifier

# создадим uplift-дерево
uplift_model = UpliftTreeClassifier(
    max_depth=5,
    min_samples_leaf=200,
    min_samples_treatment=50,
    n_reg=100,
    evaluationFunction="ED",
    control_name="0",
)

uplift_model.fit(
    df_train[feature_cols].values,
    treatment=df_train[treatment_col].apply(str).values,
    y=df_train[target_col].values,
)

Failed to import duecredit due to No module named 'duecredit'


In [4]:
# загружаем необходимые библиотеки
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(uplift_model, f)

In [6]:
with open("model.pkl", "rb") as f:
    uplift_model = pickle.load(f)

In [8]:
import requests

import numpy as np
import pandas as pd

discount = pd.read_csv("discountuplift.csv", sep="\t")

feature_cols = [
    "recency",
    "history",
    "used_discount",
    "used_bogo",
    "is_referral",
    "zip_code_Rural",
    "zip_code_Surburban",
    "zip_code_Urban",
    "channel_Multichannel",
    "channel_Phone",
    "channel_Web",
]

# пример данных для запроса
data = {"features": discount[feature_cols].sample(1).values.tolist()}

# URL сервиса
url = "http://localhost:5000/predict"

# выполнение POST-запроса
response = requests.post(url, json=data)

# проверка ответа
if response.status_code == 200:
    print("Ответ сервера:", response.json())
else:
    print("Ошибка:", response.status_code, response.text)

Ответ сервера: {'predict': [[0.12458471760797342, 0.21666666666666667]]}
